In [3]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///opt/concourse/worker/volumes/live/cffa5efe-beec-4a37-5774-350794e43990/volume/scikit-learn_1642617139916/work


In [4]:
import sklearn

In [5]:
print(sklearn.__version__)

1.0.2


In [6]:
import pickle
import pandas as pd

In [7]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [8]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [10]:
df = read_data('data/fhv_tripdata_2021-02.parquet')

In [11]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [12]:
#printing mean predicted duration for 2021.02 FHV
import numpy as np
np.mean(y_pred)

16.191691679979066

In [13]:
from datetime import datetime

In [14]:
date = datetime(year=2021, month=2, day=1)

In [15]:
year = date.year
month = date.month

In [16]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [17]:
df['ride_id']

1                2021/02_1
2                2021/02_2
3                2021/02_3
4                2021/02_4
5                2021/02_5
                ...       
1037687    2021/02_1037687
1037688    2021/02_1037688
1037689    2021/02_1037689
1037690    2021/02_1037690
1037691    2021/02_1037691
Name: ride_id, Length: 990113, dtype: object

In [29]:
df_result = df[['ride_id']].copy()

In [30]:
df_result['predictions'] = y_pred

In [38]:
df_result.head()

,ride_id,predictions
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206


In [31]:
df_result.to_parquet(
    'df_result.parquet',
    engine='pyarrow',
    compression=None,
    index=False)

In [32]:
!ls -l | grep df_result.parquet

-rw-r--r--   1 hsuyab  staff  19711440 Jun 27 23:19 df_result.parquet


In [34]:
19711440/10**6

19.71144